In [1]:
import pandas as pd
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
import math
from operator import itemgetter
import gc

In [2]:
# Read the CSV file into a DataFrame
path = 'C:\\Users\\ritaf\\Documents\\ESTUDOS\\NOVA IMS\\3º Ano\\Text Mining\\Projeto\\new_data\\'
data = pd.read_csv(path + 'lyrics_fdist_lem_stem_2500.csv')

In [3]:
data = data[['lyrics_string_fdist', 'tag']]

In [4]:
# Create a dictionary to store data for each genre
genre_freqs = {}

# Create a list to store all tokens from all genres
all_tokens = []

# Iterate through unique genres
for genre in data['tag'].unique():
    # Create a subset of the DataFrame for the current genre
    genre_df = data.loc[data['tag'] == genre].drop(columns=['tag'])
    
    # Drop NaN values in 'lyrics_string_fdist' ISTO VAI TER DE SE TIRAR
    genre_df.dropna(subset=['lyrics_string_fdist'], inplace=True)
    
    # Join the 'lyrics_string_fdist' column and tokenize
    genre_lyrics = ' '.join(list(genre_df['lyrics_string_fdist']))
    genre_tokens = word_tokenize(genre_lyrics)
    
    # Append tokens to the list for overall frequency distribution
    all_tokens.extend(genre_tokens)

    # Calculate frequency distribution for the current genre
    genre_freq = FreqDist(genre_tokens)
    
    # Store the data in the dictionary
    genre_freqs[f'freq_{genre}'] = genre_freq

# Calculate overall frequency distribution
overall_freq = FreqDist(all_tokens)

# Add the overall frequency distribution to the dictionary
genre_freqs['freq_all'] = overall_freq

# Access the frequency distributions for each genre and the overall frequency using genre_data
# For example: genre_data['freq_pop'], genre_data['freq_rap'], genre_data['overall_freq'], etc.


In [40]:
# ESTA PODE TER PALAVRAS REPETIDAS


# def log_ratio(genre_freqs, n=20):
#     # Get the overall frequency distribution for all genres
#     all_freq = genre_freqs['freq_all']

#     # Create a dictionary to store the log ratios for each genre
#     genre_log_ratios = {}

#     for genre, genre_freq in genre_freqs.items():
#         # Skip the overall frequency distribution
#         if genre == 'freq_all':
#             continue

#         # Calculate total number of words in genre and in all genres
#         total_genre_words = genre_freq.N()
#         total_all_words = all_freq.N()

#         # Calculate the genre's 100 top words
#         genre_freq_100 = genre_freq.most_common(100)

#         # Calculate log ratios for the 100 top words
#         log_ratios = {
#             word: math.log(((freq + 1) / (total_genre_words + 1)) / ((all_freq[word] + 1) / (total_all_words + 1)))
#             for word, freq in genre_freq_100
#         }

#         # Sort log ratios
#         sorted_log_ratios = sorted(log_ratios.items(), key=itemgetter(1), reverse=True)

#         # Store the top log ratios for the genre
#         genre_log_ratios[genre] = sorted_log_ratios[:n]

#     return genre_log_ratios

# # Assuming genre_freqs is a dictionary with frequency distributions for each genre
# # (including an entry with key 'overall_freq' for the overall frequency distribution)
# # Example usage:
# top_words = log_ratio(genre_freqs)

# # Print top words for each genre
# for genre, words in top_words.items():
#     print(f"Top words for {genre}: {words}")


In [41]:
# ESTA GARANTE QUE AS PALAVRAS SÃO ÚNICAS

# def log_ratio(genre_freqs, n=20):
#     # Get the overall frequency distribution for all genres
#     all_freq = genre_freqs['freq_all']

#     # Create a dictionary to store the log ratios for each genre
#     genre_log_ratios = {}

#     # Create a set to keep track of selected words across genres
#     selected_words_set = set()

#     total_all_words = all_freq.N()

#     for genre, genre_freq in genre_freqs.items():
#         # Skip the overall frequency distribution
#         if genre == 'freq_all':
#             continue

#         # Calculate total number of words in genre and in all genres
#         total_genre_words = genre_freq.N()

#         # Calculate the genre's top words
#         genre_freq_top = genre_freq.most_common(n*5)

#         # Calculate log ratios for the top words
#         log_ratios = {
#             word: math.log(((freq + 1) / (total_genre_words + 1)) / ((all_freq[word] + 1) / (total_all_words + 1)))
#             for word, freq in genre_freq_top
#         }

#         # Sort log ratios
#         sorted_log_ratios = sorted(log_ratios.items(), key=itemgetter(1), reverse=True)

#         # Select the top words that are not already selected
#         selected_words = []
#         for word, ratio in sorted_log_ratios:
#             if word not in selected_words_set:
#                 selected_words.append((word, ratio))
#                 selected_words_set.add(word)

#                 # Break once n unique words are selected for the genre
#                 if len(selected_words) == n:
#                     break

#         # Store the top log ratios for the genre
#         genre_log_ratios[genre] = selected_words

#     return genre_log_ratios


In [21]:
genre_percentages = (data['tag'].value_counts() / len(data))
genre_percentages_dict = genre_percentages.to_dict()

In [27]:
# ESTE GRANATE QUE PALÁVRAS SÃO ÚNICAS E QUE O NÚMERO DE PALAVRAS É PROPORCIONAL AO TAMANHO DO GÉNERO

def log_ratio(genre_freqs, genre_percentages, total_words=100):
    # Get the overall frequency distribution for all genres
    all_freq = genre_freqs['freq_all']

    # Create a dictionary to store the log ratios for each genre
    genre_log_ratios = {}

    # Create a set to keep track of selected words across genres
    selected_words_set = set()

    total_all_words = all_freq.N()

    for genre, genre_freq in genre_freqs.items():
        # Skip the overall frequency distribution
        if genre == 'freq_all':
            continue

        # Get the percentage of representativeness for the current genre
        genre_percentage = genre_percentages.get(genre.split('_')[1])

        # Calculate the number of words to select for the genre based on its representativeness
        words_to_select = int(round(genre_percentage * total_words))

        # Calculate the genre's top words
        genre_freq_top = genre_freq.most_common(words_to_select * 5)

        # Calculate log ratios for the top words
        log_ratios = {
            word: math.log(((freq + 1) / (genre_freq.N() + 1)) / ((all_freq[word] + 1) / (total_all_words + 1)))
            for word, freq in genre_freq_top
        }

        # Sort log ratios
        sorted_log_ratios = sorted(log_ratios.items(), key=itemgetter(1), reverse=True)

        # Select the top words that are not already selected
        selected_words = []
        for word, ratio in sorted_log_ratios:
            if word not in selected_words_set:
                selected_words.append((word, ratio))
                selected_words_set.add(word)

                # Break once the required number of unique words are selected for the genre
                if len(selected_words) == words_to_select:
                    break

        # Store the top log ratios for the genre
        genre_log_ratios[genre] = selected_words

    return genre_log_ratios


In [28]:
# Finding the top words
top_words = log_ratio(genre_freqs, genre_percentages = genre_percentages_dict, total_words = 100)

# Taking a look at the top words for each genre
for genre, words in top_words.items():
    print(f"Top words for {genre}: {words}")

Top words for freq_rock: [('dead', 0.6152011021884276), ('inside', 0.6009252919273419), ('prechorus', 0.5809816617226533), ('away', 0.5677597721397807), ('end', 0.5490948272168785), ('fire', 0.5401145592169599), ('bridge', 0.5303989700415128), ('eyes', 0.501319821385536), ('die', 0.49638555698730513), ('alone', 0.4772915658969056), ('lost', 0.46496344335943296), ('light', 0.46400376130804455), ('fall', 0.44572787037487166), ('left', 0.44115071178242765), ('chorus', 0.43491511560080504), ('nothing', 0.4237892360098731), ('ive', 0.4237679626182387), ('soul', 0.40175160845654595), ('world', 0.3670590484450102)]
Top words for freq_rap: [('niggas', 0.8389884134636949), ('nigga', 0.8297372082321213), ('hoes', 0.8104885055811865), ('bitch', 0.8012863055273524), ('bitches', 0.7993400219441436), ('lil', 0.7847508112425773), ('rap', 0.7786411427992727), ('ayy', 0.7747667807107773), ('shit', 0.743500958318561), ('fuckin', 0.7320383340748741), ('aye', 0.7297315810718064), ('ima', 0.714889561997828

In [26]:
words_only = [word for sublist in top_words.values() for word, _ in sublist]
unique_word_count = len(set(words_only))
print("Number of unique words:", unique_word_count)
print(words_only)

Number of unique words: 101
['dead', 'inside', 'prechorus', 'away', 'end', 'fire', 'bridge', 'eyes', 'die', 'alone', 'lost', 'light', 'fall', 'left', 'chorus', 'nothing', 'ive', 'soul', 'world', 'niggas', 'nigga', 'hoes', 'bitch', 'bitches', 'lil', 'rap', 'ayy', 'shit', 'fuckin', 'aye', 'ima', 'tryna', 'fuck', 'gon', 'hook', 'ass', 'imma', 'aint', 'yea', 'hit', 'money', 'bout', 'fucking', 'cuz', 'damn', 'pull', 'top', 'intro', 'na', 'sing', 'tonight', 'falling', 'dance', 'forever', 'someone', 'love', 'heart', 'sweet', 'sun', 'cry', 'ooh', 'rain', 'together', 'waiting', 'hey', 'sky', 'dream', 'night', 'touch', 'alright', 'forget', 'baby', 'comes', 'maybe', 'dreams', 'mine', 'close', 'find', 'believe', 'home', 'ill', 'true', 'feel', 'hold', 'blue', 'song', 'wish', 'come', 'shall', 'verse', 'way', 'back', 'girl', 'yeah', 'need', 'know', 'want', 'upon', 'may', 'said', 'would']


In [60]:
data['lyrics_string_fdist'].value_counts()

tell would like lyrics song make find lyrics first help lyrics song would great help                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [63]:
data_lyr = data['lyrics_string_fdist'].dropna()

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
#data_lyr = data.lyrics_string_fdist

def count_vectorizer_to_df(train_lst_strs):

    # Create an instance of the CountVectorizer class - Default vectorizer does not remove stop words
    vectorizer = CountVectorizer(vocabulary=words_only)

    # Fit the vectorizer to the text data and transform the text data into a frequency matrix
    frequency_matrix = vectorizer.fit_transform(train_lst_strs)


    # Convert the frequency matrix to a Pandas DataFrame
    train_df = pd.DataFrame(frequency_matrix.toarray(), columns=vectorizer.get_feature_names_out())

    return train_df

train_df = count_vectorizer_to_df(data_lyr).dropna()
train_df

,solo,instrumental,flesh,shadows,bones,darkness,burn,fate,burning,souls,...,near,strength,child,beauty,strange,power,read,kept,point,sense
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134952,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134953,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
134954,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134955,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
